# CIwA Voting Methods Tutorial

This notebook explains and demonstrates the different VotingMethods available in the CIwA module. We will cover the LabelVotingMethod and CompareVotingMethod, showcasing different classes for each:
1. YesNoLabel
2. EnumLabel
3. ScoreLabel
4. ScoreCompare
5. RankingCompare

In [ ]:
# Prepare correct module path for notebook
import sys
import os

# Add the parent directory of the 'ciwa' module to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Setup

First, let's import the necessary modules and initialize the CIwA process.

In [ ]:
import asyncio
import ciwa

config_dict = {
    "process": {
        "name": "CIwA VotingMethods Demonstration",
        "description": "A demo process for CIwA VotingMethods",
        "default_session_settings": {
            "submissions_per_participant_per_topic": 3, # number of submissions a participant can submit per topic
        },
    }
}

# Initialize configuration manager
config_manager = ciwa.ConfigManager(config=config_dict)

# Create process configuration
process_config = config_manager.get_config("process")
process = ciwa.ProcessFactory.create_process(process_config)

print(f"Process {process.name} initialized with UUID: {process.uuid}")

### Let's use a function to create a session config for a given VotingMethod.

We will use this to make it easy to spin up new sessions with the same group of Participant agents for each VotingMethod. It takes a `title` and `description` for a single Topic that Participants will be prompted for submissions on.

Use `LLMAgentParticipant` to generate dummy data without waiting on API calls and using up tokens, or `ConversableAgentParticipant` to call whatever models are saved in your `OAI_CONFIG_LIST.json`.

In [ ]:
USE_DUMMY_DATA = True
participant_type = "LLMAgentParticipant" if USE_DUMMY_DATA else "ConversableAgentParticipant"

In [ ]:
number_of_participants = 4

def create_session_config(voting_method, title, description, voting_method_args={}):
    voting_method_config = {"type": voting_method}
    voting_method_config.update(voting_method_args)
    new_session_config = {
        "name": f"{voting_method} Demo Session",
        "description": f"A session demonstrating the use of {voting_method}",
        "topics": [
            {
                "title": title,
                "description": description,
                "voting_method": voting_method_config
            }
        ],
        "participants": [
            {
                "type": participant_type,
                "model": "gpt-3.5-turbo",
                "temperature": 1.5, # higher temperature (0-2) increases randomness/diversity of output
                "cache_seed": None, # set to None to avoid caching LLM replies
            } for _ in range(number_of_participants)
        ],
        "save_results": False    
    }
    return new_session_config

### And a function to run a session from a config one at a time

We could define all the settings for all sessions at once and then run an entire process using `process.run()`. But let's create and run one session at a time to illustrate the different VotingMethods.

In [ ]:
# Use the nest_asyncio package to allow nested use of asyncio.run() in a Notebook
import nest_asyncio
nest_asyncio.apply()

def create_and_run_session(config):    
    # Add the session to the process
    process.add_session(config)
    asyncio.run(process.run_next_session())
    return process.completed_sessions[-1]

## 1. YesNoLabel

The YesNoLabel voting method allows participants to vote 'Yes' or 'No' on each submission.

In [ ]:
# Configure a session with YesNoLabel voting method
config = create_session_config(
    "YesNoLabel",
    title="What are non-obvious principles you think an AI Chat Assistant should follow that not everyone might agree with?",
    description="There are probably many principles for an AI Chat Assistant that almost everyone will agree with. "\
                "Try to only submit a principle that you think an AI Chat Assistant should follow, but may be controversial to other people."
)

session = create_and_run_session(config)

In [ ]:
# Visualize the results
ciwa.display_results(session, aggregated_only=True)

## 2. EnumLabel

The EnumLabel voting method allows participants to choose from a predefined set of labels.

In [ ]:
# Configure a session with EnumLabel voting method
config = create_session_config(
    "EnumLabel",
    title="What are some challenging scenarios an AI Chat Assistant might face when prompted by a user?",
    description="Describe a very specific and realistic prompt from a human user to an AI Chat Assistant. "\
                "Describe four possible realistic responses the AI Chat Assistant might make, with each response enumerated A. B. C. D.",
    voting_method_args={"enum_values": ["A", "B", "C", "D"]}    
)
session = create_and_run_session(config)

In [ ]:
# Visualize the results
ciwa.display_results(session, aggregated_only=True)

## 3. ScoreLabel

The ScoreLabel voting method allows participants to assign a score to each submission.

In [ ]:
# Configure a session with EnumLabel voting method
config = create_session_config(
    "ScoreLabel",
    title="What rules should an AI Chat Assistant follow in oder to best maximize user well-being?",
    description="Describe a rule for an AI Chat Assistant that will maximize the well-being of the user prompting the Assistant.",
    voting_method_args={
        "start_value": 1,
        "end_value": 10,
        "increment_value": 1
    }
)
session = create_and_run_session(config)

In [ ]:
# Visualize the results
ciwa.display_results(session, aggregated_only=True)

## 4. ScoreCompare

The ScoreCompare voting method allows participants to view all submissions and assign a score to each.

In [ ]:
# Configure a session with EnumLabel voting method
config = create_session_config(
    "ScoreCompare",
    title="What rules should an AI Chat Assistant follow in oder to best maximize user well-being?",
    description="Describe a rule for an AI Chat Assistant that will maximize the well-being of the user prompting the Assistant.",
    voting_method_args={
        "start_value": 1,
        "end_value": 10,
        "increment_value": 1
    }
)
session = create_and_run_session(config)

In [ ]:
# Visualize the results
ciwa.display_results(session, aggregated_only=True)

## 5. RankingCompare

The RankingCompare voting method allows participants to rank all submissions from most preferred to least preferred.

In [ ]:
# Configure a session with EnumLabel voting method
config = create_session_config(
    "RankingCompare",
    title="What kind of response from an AI Chat Asisstant would best maximize the "\
          "well-being of a human user who prompts the Assistant with the following:\n"\
          "Hi, I'm a junior employee at my company, and I recently discovered that my "\
          "boss has been altering financial records to make our department's performance "\
          "look better than it actually is. I'm worried about the legal and ethical "\
          "implications, but I'm also afraid of losing my job if I report this. What "\
          "should I do?",
    description="Provide a realistic response as an AI Chat Assistant that will best "\
          "maximize the well-being of a human user who gives you this prompt.",
    voting_method_args={}
)
session = create_and_run_session(config)

In [ ]:
ciwa.display_results(session, aggregated_only=True)